This is a simplified version of the `aoi-2-dataset` notebook that specifically streamlines collecting imagery for HydroSHED geometries by HYBAS_ID. 

In [ ]:
import ee
import geemap
from datetime import datetime
from pathlib import Path

In [ ]:
try:
    ee.Initialize()
except: 
    ee.Authenticate()
    ee.Initialize()

You can collapse the "Functions" header and run the cells while collapsed

# Functions (collapse)

## Original aoi-2-dataset function

In [ ]:
def maskS2clouds(image):
        """Masks clouds in S2 images

    Parameters:
    image (Image): A single Image in an ImageCollection or standalone Image

    Returns:
    Image with masked features and original metadata

    """
        qa = image.select('QA60')

        # Bits 10 and 11 are clouds and cirrus, respectively.
        cloudBitMask = 1 << 10
        cirrusBitMask = 1 << 11

        # Both flags should be set to zero, indicating clear conditions.
        mask = qa.bitwiseAnd(cloudBitMask).eq(0) \
            .And(qa.bitwiseAnd(cirrusBitMask).eq(0))

        return image.updateMask(mask) \
            .divide(10000) \
            .copyProperties(image, ['system:time_start'])

def clp(image):
    '''Clips a single Image to a region of interest'''
    return image.clip(aoi)

def mosaicByDate(imcol):
        """Creates a mosaicked Image for a single date if there are
        multiple images from a single date 

    Parameters:
    imcol (ImageCollection): An ImageCollection with images from one or more dates

    Returns:
    ImageCollection with images mosaicked by date

        """
        # Get a list of unique dates in the image collection
        imlist = imcol.toList(imcol.size())

        unique_dates = imlist.map(lambda im: ee.Image(im).date().format("YYYY-MM-dd")).distinct().getInfo()

        # Create an empty list to store mosaic images
        mosaic_imlist = []

        # Loop through unique dates and create mosaic images
        for date_str in unique_dates:
            date = ee.Date.parse("YYYY-MM-dd", date_str)
            mosaic_image = imcol.filterDate(date, date.advance(1, "day")).mosaic()
            mosaic_image = mosaic_image.set("system:time_start", date.millis(), "system:id", date_str)
            mosaic_imlist.append(mosaic_image)

        return ee.ImageCollection(mosaic_imlist)

def addNDVI(image):
  '''Adds S2's NDVI band to each image (in an ImageCollection)'''
  ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')
  return image.addBands(ndvi)

def addNDSI(image):
    '''Adds S2's NDSI band to each image (in an ImageCollection)'''
    ndsi = image.normalizedDifference(['B3', 'B11']).rename('NDSI')
    return image.addBands(ndsi)

def addElevation(image):
  '''Adds ArcticDEM elevation to each image (in an ImageCollection)'''
  elevation = ee.Image("UMN/PGC/ArcticDEM/V3/2m_mosaic").select('elevation').clip(aoi).rename('ArcticDEM')
  return image.addBands(elevation)

def get_mean_snow_cover(image):
        """Adds a value for scene-averaged MODIS-dervied snow cover to an image (in an ImageCollection)

    Parameters:
    image (Image): A single Image in an ImageCollection or standalone Image

    Returns:
    Image with snow cover mean as a band

        """    
        # Get MODIS snow cover product for day and location
        ndsi_image = ee.ImageCollection('MODIS/061/MOD10A1').filterDate(
            image.date(), image.date().advance(1, 'day')).first().select('NDSI_Snow_Cover').clip(aoi)
        
        image = image.addBands(ndsi_image)

        # Get mean value across the scene 
        mean_value = image.reduceRegion(
            reducer=ee.Reducer.mean(),
            geometry=aoi,
            scale=100,  # Resolution of Landsat data in meters
            maxPixels = 1e9
        )

        # Get the mean value for the band
        mean_band_value = mean_value.get('NDSI_Snow_Cover')

        # Set the mean value as an image property
        return image.set("mean_" + 'NDSI_Snow_Cover', mean_band_value)

def calculateNoDataPercentage(image):
        """Add data on masked pixel percentage as a band
        note that total_pixels needs to be calculated first

    Parameters:
    image (Image): A single Image in an ImageCollection or standalone Image

    Returns:
    Image with "nodata_percentage band added"

    """
    
    # Any masked no data stuff will be equal to 1
        nodata_mask = image.select('B1').mask().eq(0)
            
        # Sum up the nodata 1's
        # nodata_pixels = nodata_mask.reduceRegion(
        #     reducer=ee.Reducer.sum(),
        #     geometry=aoi,
        #     scale=100,  # s2 res
        #     maxPixels = 1e9
        # )
        
        # # Calculate the percentage of NoData values
        # percentage_nodata = nodata_pixels.getNumber('B1').divide(total_pixels.getNumber('B1')).multiply(100)

        
        # # Set the NoData percentage as an image property
        # return image.set("nodata_percentage", percentage_nodata)  
        nodata_stats = nodata_mask.reduceRegion(
        reducer=ee.Reducer.sum().combine(
            reducer2=ee.Reducer.count(),
            sharedInputs=True
        ),
        geometry=aoi,  # Assuming 'aoi' is defined somewhere in your script
        scale=100,  # Adjust the scale according to your requirements
        maxPixels=1e9
        )

        # Extract sum and count values
        sum_nodata_pixels = nodata_stats.getNumber('B1_sum')
        count_nodata_pixels = nodata_stats.getNumber('B1_count')

        # Calculate percentage
        nodata_percentage = sum_nodata_pixels.divide(count_nodata_pixels).multiply(100)

        # Add NoData percentage as a band
        return image.set('nodata_percentage', nodata_percentage)


def get_white_pixel_percent(image):
        """Add data on pixel percentage that is white in grayscale as a band
        note that total_pixels needs to be calculated first

    Parameters:
    image (Image): A single Image in an ImageCollection or standalone Image

    Returns:
    Image with "white_percentage" band added

    """    
        grayscale = image.expression(
            '(.3 * 1e4 * R) + (.59 * 1e4 * G) + (.11 * 1e4 * B)', {
            # '(R + G + B) / 3', {
            'R': image.select('B4'),
            'G': image.select('B3'),
            'B': image.select('B2')
        })

        white_mask = grayscale.gt(2000)
        
        # white_mask needs to = 1

        # white_pixels = white_mask.reduceRegion(
        #     reducer=ee.Reducer.sum(),
        #     geometry=aoi,
        #     scale=100,  # s2 res
        #     maxPixels = 1e9
        # )

        # # # Calculate the total number of pixels within the ROI
        # # total_pixels = image.select('B1').reduceRegion(
        # #     reducer=ee.Reducer.count(),
        # #     scale=10,  # s2 res
        # #     maxPixels = 1e9
        # # )

        # # percentage_white = white_pixels.getNumber('constant').divide(total_pixels.getNumber('B1')).multiply(100)
        
        # total_pixels = image.getNumber('total_pixels')  # Get total_pixels from the image properties

        # percentage_white = white_pixels.getNumber('constant').divide(total_pixels).multiply(100)
        
        # # Set the NoData percentage as an image property
        # return image.set("white_percentage", percentage_white).set("white_pixel_count", white_pixels.getNumber('constant')) 
        white_pixel_stats = white_mask.reduceRegion(
        reducer=ee.Reducer.sum().combine(
                reducer2=ee.Reducer.count(),
                sharedInputs=True
            ),
            geometry=aoi,  # Assuming 'aoi' is defined somewhere in your script
            scale=100,  # Adjust the scale according to your requirements
            maxPixels=1e9
        )

        # Extract sum and count values
        sum_white_pixels = white_pixel_stats.getNumber('constant_sum')
        count_white_pixels = white_pixel_stats.getNumber('constant_count')

        # Calculate percentage
        white_percentage = sum_white_pixels.divide(count_white_pixels).multiply(100)

        return image.set("white_percentage", white_percentage)

def calcTotalPixels(image):
    """Add data on total pixels as a band

    Parameters:
    image (Image): A single Image in an ImageCollection or standalone Image

    Returns:
    Image with "total_pixels" band added

    """    
    total_pixels = image.select('B1').reduceRegion(
        reducer=ee.Reducer.count(),
        scale=100,  
        maxPixels = 1e9,
        geometry=aoi
    )
    return image.set("total_pixels", total_pixels.getNumber('B1')) 

def getVisibleImages(Map):
        """Retrieves names of layers visible on the Map 

    Parameters:
    Map (Map): A geemap.Map() 

    Returns:
    A list of strings corresponding to the labels on the Map layers
    if they are dates (as needed for the original notebook)

    """    
        map_layers = list(Map.layers)
        visibility_status = [layer.visible for layer in map_layers]
        visible_layers = [x.name for x, y in zip(map_layers, visibility_status) if y == True]
        return [x for x in visible_layers if '-' in x and datetime.strptime(x, '%Y-%m-%d')]


## New aoi-2-dataset-lite wrapper function

In [ ]:
def get_images(aoi,
    snow_cover_threshold = 0,
    threshold_nodata_percent = 0.5,
    threshold_white_percent = 2,
    image_limit=5,
    ):

    longitude = aoi.geometry().centroid().coordinates().get(0).getInfo()
    latitude = aoi.geometry().centroid().coordinates().get(1).getInfo()

    dataset = (
                    ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
                    .filter(ee.Filter.calendarRange(2019,2023,'year'))
                    .filter(ee.Filter.calendarRange(5,9,'month'))
                    # Pre-filter to get less cloudy granules.
                    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10))
                    # .filterBounds(aoi.centroid())'
                    .filterBounds(aoi)
                    .map(clp)
                    .map(maskS2clouds)
    )

    dataset = mosaicByDate(dataset).map(addNDVI).map(addNDSI).map(addElevation)

    collection_with_data = dataset.map(calcTotalPixels).map(calculateNoDataPercentage).map(get_white_pixel_percent)

    collection_with_data_size = len(collection_with_data.aggregate_array("system:index").getInfo())
    print(f'Number of images in initial collection: {collection_with_data_size}')

    # Basically a "too many aggregated requests" error can be thrown if you do TOO much at once (like three reduceRegions and filters)
    # So my strategy here is to subsume the snow filter into the white filter

    filtered_collection = collection_with_data.filter(ee.Filter.lte("nodata_percentage", threshold_nodata_percent))
    filtered_collection = filtered_collection.filter(ee.Filter.lte("white_percentage", threshold_white_percent))

    filtered_collection_size = len(filtered_collection.aggregate_array("system:index").getInfo())

    print(f'Number of images in collection: {filtered_collection_size}')

    # This line only instantiates a "Map" object from gee
    # You need to make one to add layers to it
    # but we don't display it yet 

    # If you run this line after adding layers, 
    # you will lose the layers because you made a new Map
    Map = geemap.Map(center=[latitude, longitude], zoom=11)

    for image_id in filtered_collection.aggregate_array("system:index").getInfo()[0:image_limit]:
        image = filtered_collection.filterMetadata("system:index", "equals", image_id).first()
        
        image_RGB = image.select('B4', 'B3', 'B2') 
        # image_RGB = image.select('B4') 
        RGB_vis_params = {'min': 0.0, 'max': 0.3}
        Map.addLayer(image_RGB, RGB_vis_params, ee.Image(image).date().format('yyyy-MM-dd').getInfo(), True)

    print(f'Remember I am limiting the image list to {image_limit} of {filtered_collection_size} images')

    return Map, filtered_collection

# Run me!

In [ ]:
# Input the ID as an integer
HYBAS_ID = 8100369270
aoi = ee.FeatureCollection("WWF/HydroSHEDS/v1/Basins/hybas_10").filter(ee.Filter.eq('HYBAS_ID', HYBAS_ID))

# You can give the function parameter values different from the defaults
Map, filtered_collection = get_images(aoi, image_limit=3)

Map

Now if you click on the wrench in the right top corner, followed by the "Layers" button, you can toggle through imagery that passed the filter. 

If you notice squares missing from the imagery, it might be a rendering/tiling issue - if you zoom in and out the map will re-tile and the problem is usually solved. The data are there, I promise. 


Now you can download any imagery you want by keeping that layer visible and running the following script

In [ ]:
aoi_name = str(HYBAS_ID)

# If you want to save things locally
aoi_path = Path('.', str(aoi_name))

# If you're in the arctic group!
# aoi_path = Path('/sciclone/data10/watersheds', str(aoi_name))

Path(Path(aoi_path)).mkdir(parents=True, exist_ok=True)

In [ ]:
date_layers = getVisibleImages(Map)

for date_string in date_layers:
    print(date_string)

    imageDate = ee.Date(date_string)

    to_download = filtered_collection.filterDate(imageDate).first().visualize(
        bands=['B4', 'B3', 'B2'],
        min=0.001, max=0.3
        )

    if isinstance(aoi, ee.featurecollection.FeatureCollection):
        geemap.ee_to_shp(((aoi)), filename=str(aoi_path.joinpath(f'{aoi_name}.shp')))
    
        geemap.download_ee_image(to_download, str(aoi_path.joinpath(f'{aoi_name}_{date_string}_RGB.tif')), scale=10, region=aoi.geometry(), crs='EPSG:3995')

        to_download = filtered_collection.filterDate(imageDate).first().select('NDVI')

        geemap.download_ee_image(to_download, str(aoi_path.joinpath(f'{aoi_name}_{date_string}_NDVI.tif')), scale=10, region=aoi.geometry(), crs='EPSG:3995')

        to_download = filtered_collection.filterDate(imageDate).first().select('ArcticDEM')

        geemap.download_ee_image(to_download, str(aoi_path.joinpath(f'{aoi_name}_ArcticDEM.tif')), scale=2, region=aoi.geometry(), crs='EPSG:3995')

    else:
        print("Your AOI is not a FeatureCollection, using backup routine.")

        geemap.ee_to_shp(ee.FeatureCollection(ee.Feature(ee.Geometry(aoi))), filename=str(aoi_path.joinpath(f'{aoi_name}.shp')))
  
        geemap.download_ee_image(to_download, str(aoi_path.joinpath(f'{aoi_name}_{date_string}_RGB.tif')), scale=10, region=aoi, crs='EPSG:3995')

        to_download = filtered_collection.filterDate(imageDate).first().select('NDVI')

        geemap.download_ee_image(to_download, str(aoi_path.joinpath(f'{aoi_name}_{date_string}_NDVI.tif')), scale=10, region=aoi, crs='EPSG:3995')

        to_download = filtered_collection.filterDate(imageDate).first().select('ArcticDEM')

        geemap.download_ee_image(to_download, str(aoi_path.joinpath(f'{aoi_name}_ArcticDEM.tif')), scale=2, region=aoi, crs='EPSG:3995')
